In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.listdir('/content/drive/MyDrive')

['Colab Notebooks',
 'Fiecare stea se luptă cu gravitația.docx',
 'admissionletter.pdf',
 'DSCF1654.JPG',
 'DSCF1660.JPG',
 'DSCF1655.JPG',
 'DSCF1666.JPG',
 'DSCF1668.JPG',
 'DSCF1669.JPG',
 'DSCF1672.JPG',
 'DSCF1673.JPG',
 'cv_2025.pdf',
 'constellation_dataset.zip',
 'best_constellation_model.pt',
 'processed_image_grayscale.jpg',
 'processed_image_scorpius.jpg',
 'best_gas_tank_model.pth',
 'training_history.png',
 'top_cropped_label.zip',
 'front_cropped_label.zip',
 'back_left_cropped_label.zip',
 'back_right_cropped_label.zip']

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import zipfile
from pathlib import Path
from sklearn.model_selection import train_test_split

# Configuration - zip files are in Drive root
drive_data_path = Path('/content/drive/MyDrive')  # <-- UPDATED PATH
zip_files = [
    'back_left_cropped_label.zip',
    'back_right_cropped_label.zip',
    'front_cropped_label.zip',
    'top_cropped_label.zip'
]

# Working directory in Colab (faster than Drive)
base_path = Path('/content/data')
train_ratio = 0.8

# Create base directory
base_path.mkdir(parents=True, exist_ok=True)

# Unzip all files
print("Unzipping files...")
for zip_file in zip_files:
    zip_path = drive_data_path / zip_file
    if zip_path.exists():
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            extract_to = base_path / zip_path.stem
            zip_ref.extractall(extract_to)
            print(f"  ✓ Extracted: {zip_file}")
    else:
        print(f"  ✗ Warning: {zip_file} not found at {zip_path}")

# Now let's debug what was extracted
print("\n" + "="*60)
print("Checking extracted directory structure...")
print("="*60)

sources = [
    'back_left_cropped_label',
    'back_right_cropped_label',
    'front_cropped_label',
    'top_cropped_label'
]

for source in sources:
    source_dir = base_path / source
    print(f"\n📁 {source}:")
    print(f"   Exists: {source_dir.exists()}")

    if source_dir.exists():
        print(f"   Contents:")
        for item in sorted(source_dir.iterdir()):
            if item.is_dir():
                file_count = len(list(item.iterdir()))
                print(f"     📂 {item.name}/ ({file_count} files)")
            else:
                print(f"     📄 {item.name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unzipping files...
  ✓ Extracted: back_left_cropped_label.zip
  ✓ Extracted: back_right_cropped_label.zip
  ✓ Extracted: front_cropped_label.zip
  ✓ Extracted: top_cropped_label.zip

Checking extracted directory structure...

📁 back_left_cropped_label:
   Exists: True
   Contents:
     📄 classes.txt
     📂 images/ (500 files)
     📂 labels/ (500 files)
     📄 notes.json

📁 back_right_cropped_label:
   Exists: True
   Contents:
     📄 classes.txt
     📂 images/ (500 files)
     📂 labels/ (500 files)
     📄 notes.json

📁 front_cropped_label:
   Exists: True
   Contents:
     📄 classes.txt
     📂 images/ (503 files)
     📂 labels/ (503 files)
     📄 notes.json

📁 top_cropped_label:
   Exists: True
   Contents:
     📄 classes.txt
     📂 images/ (504 files)
     📂 labels/ (504 files)
     📄 notes.json


In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import zipfile
from pathlib import Path
from sklearn.model_selection import train_test_split

# Configuration
drive_data_path = Path('/content/drive/MyDrive')
zip_files = [
    'back_left_cropped_label.zip',
    'back_right_cropped_label.zip',
    'front_cropped_label.zip',
    'top_cropped_label.zip'
]

base_path = Path('/content/data')
train_ratio = 0.8

# Create base directory
base_path.mkdir(parents=True, exist_ok=True)

# Unzip all files
print("Unzipping files...")
for zip_file in zip_files:
    zip_path = drive_data_path / zip_file
    if zip_path.exists():
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            extract_to = base_path / zip_path.stem
            zip_ref.extractall(extract_to)
            print(f"  ✓ Extracted: {zip_file}")

# Source directories
sources = [
    'back_left_cropped_label',
    'back_right_cropped_label',
    'front_cropped_label',
    'top_cropped_label'
]

# Create combined dataset structure
combined_path = base_path / 'combined'
for split in ['train', 'val']:
    (combined_path / 'images' / split).mkdir(parents=True, exist_ok=True)
    (combined_path / 'labels' / split).mkdir(parents=True, exist_ok=True)

# Collect all image-label pairs
print("\nCollecting image-label pairs...")
all_files = []

for source in sources:
    img_dir = base_path / source / 'images'
    lbl_dir = base_path / source / 'labels'

    if img_dir.exists() and lbl_dir.exists():
        # Get all image files
        images = (list(img_dir.glob('*.jpg')) +
                 list(img_dir.glob('*.JPG')) +
                 list(img_dir.glob('*.png')) +
                 list(img_dir.glob('*.jpeg')))

        for img in images:
            # Find corresponding label file
            label_file = lbl_dir / (img.stem + '.txt')
            if label_file.exists():
                all_files.append((source, img, label_file))

print(f"\nTotal image-label pairs found: {len(all_files)}")
for source in sources:
    count = sum(1 for s, _, _ in all_files if s == source)
    print(f"  {source}: {count}")

# Split into train/val
train_files, val_files = train_test_split(
    all_files,
    train_size=train_ratio,
    random_state=42,
    shuffle=True
)

print(f"\nSplit: Train={len(train_files)}, Val={len(val_files)}")

# Copy files to combined dataset
def copy_files(file_list, split):
    for i, (source, img_path, lbl_path) in enumerate(file_list):
        # Simplify source name for prefix
        prefix = source.replace('_cropped_label', '')

        # Create unique filename with source prefix
        new_img_name = f"{prefix}_{img_path.name}"
        new_lbl_name = f"{prefix}_{img_path.stem}.txt"

        # Copy image
        shutil.copy(img_path, combined_path / 'images' / split / new_img_name)

        # Copy label
        shutil.copy(lbl_path, combined_path / 'labels' / split / new_lbl_name)

        if (i + 1) % 100 == 0:
            print(f"  Copied {i + 1}/{len(file_list)} files...")

print("\nCopying training files...")
copy_files(train_files, 'train')

print("Copying validation files...")
copy_files(val_files, 'val')

print(f"\n✓ Combined dataset created at: {combined_path}")
print(f"  Train images: {len(list((combined_path / 'images' / 'train').glob('*')))}")
print(f"  Val images: {len(list((combined_path / 'images' / 'val').glob('*')))}")

# Create data.yaml
yaml_content = f"""path: {combined_path}
train: images/train
val: images/val

nc: 1
names: ['gas_bottle']
"""

yaml_path = combined_path / 'data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"\n✓ data.yaml created")
print("✓ Dataset ready for training!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unzipping files...
  ✓ Extracted: back_left_cropped_label.zip
  ✓ Extracted: back_right_cropped_label.zip
  ✓ Extracted: front_cropped_label.zip
  ✓ Extracted: top_cropped_label.zip


Total image-label pairs found: 2007
  back_left_cropped_label: 500
  back_right_cropped_label: 500
  front_cropped_label: 503
  top_cropped_label: 504

Split: Train=1605, Val=402

Copying training files...
  Copied 100/1605 files...
  Copied 200/1605 files...
  Copied 300/1605 files...
  Copied 400/1605 files...
  Copied 500/1605 files...
  Copied 600/1605 files...
  Copied 700/1605 files...
  Copied 800/1605 files...
  Copied 900/1605 files...
  Copied 1000/1605 files...
  Copied 1100/1605 files...
  Copied 1200/1605 files...
  Copied 1300/1605 files...
  Copied 1400/1605 files...
  Copied 1500/1605 files...
  Copied 1600/1605 files...
Copying validation files...
  Copied 100/4

In [8]:
# Install ultralytics
!pip install ultralytics -q

from ultralytics import YOLO
import torch

# Check GPU
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Load YOLOv11n
model = YOLO('yolo11n.pt')

# Train
results = model.train(
    data='/content/data/combined/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    patience=20,
    save=True,
    plots=True,
    name='gas_bottle_detection',
    cache=True
)

# Validate
metrics = model.val()
print(f"\n✓ Training complete!")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
GPU Available: True
GPU: Tesla T4
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/combined/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, f

In [9]:
# Save models back to Google Drive
runs_path = Path('/content/runs/detect/gas_bottle_detection')
drive_output = Path('/content/drive/MyDrive/gas_bottle_yolo_models')
drive_output.mkdir(parents=True, exist_ok=True)

# Copy weights and results
shutil.copy(runs_path / 'weights' / 'best.pt', drive_output / 'best.pt')
shutil.copy(runs_path / 'weights' / 'last.pt', drive_output / 'last.pt')

for plot in ['results.png', 'confusion_matrix.png', 'val_batch0_pred.jpg']:
    if (runs_path / plot).exists():
        shutil.copy(runs_path / plot, drive_output / plot)

print(f"✓ Models saved to: {drive_output}")

✓ Models saved to: /content/drive/MyDrive/gas_bottle_yolo_models
